In [5]:
# 各種オプティマイザの特徴と実装方法

# Adam最適化が推奨だが、AdaGrad、RMSProp、Adam最適化は一部データセットにおいて汎化性能が低いことが指摘されている
# なのでAdam最適化ｄ汎化性能を検証するか、NAG を使用することがオプティマイザの選択肢となる


# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [ ]:
# Momentum最適化:勾配を速度ではなく加速度として扱うことで収束速度を速めている
# momentum= の値は0.9 固定で問題なし
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9)

In [ ]:
# NAG : ネステロフ慣性最適化という慣性が働く方向とは少し先の勾配をはかる。momentum最適化よりさらに高速
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9, use_nesterov=True)

In [ ]:
# AdaGrad : 最も急な次元に沿って勾配ベクトルをスケールダウンする
# つまり学習率を下げるが傾斜が急な次元では傾斜が緩やかな次元よりも早く学習率を下げることで全体の最適値に近い値に更新する
# 深いNNでは早期に学習が止まってしまう傾向があるので線形回帰など単純なものにつかうこと
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)

In [ ]:
# RMSProp : AdaGradの欠点解消版。全体最適値に収束しない(早期で止まる)を防止するため訓練の最初だけ勾配ベクトルをスローダウンさせている
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate,
                                      momentum=0.9, decay=0.9, epsilon=1e-10)

In [ ]:
# Adam最適化 : Momentum最適化 + RMSProp
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

In [10]:
# MNIST SELU　で試してみる
import tensorflow as tf

reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1") # seluがあるよ！
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2") # seluがあるよ！！
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
#learning_rate = 0.01
# ここのオプティマイザを変える　オプティマイザによっては学習率設定が不要になったり、追加パラメータが必要になる
# 学習率スケジューリングを見るためコメント化
#with tf.name_scope("train"):
#    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
#    training_op = optimizer.minimize(loss)

# 学習率のスケジューリング
# Momentmu最適化やNAGの場合に有効な方法。Adam最適ななどは自動で学習率を下げていくので考慮不要
# 訓練中に学習率を減衰させていく
# いくつか方法があるが、実装のしやすさと調整のしやすさ、最適解への収束速度から指数関数的スケジューリングを推奨
with tf.name_scope("train"):       # not shown in the book
    initial_learning_rate = 0.1 # 初期学習率
    decay_steps = 10000 # 10,000ステップごとに学習率を減少させる設定
    decay_rate = 1/10 # 減少させていく学習率
    global_step = tf.Variable(0, trainable=False, name="global_step") # 現在の訓練のイテレーション番号を管理する
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, # 指数関数的減衰学習率の定義
                                               decay_steps, decay_rate)
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9) # 例としてMomentum最適化を使っている
    training_op = optimizer.minimize(loss, global_step=global_step) # ここにglobal_stepを設定している

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()
n_epochs = 5
batch_size = 50

In [7]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [8]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [12]:
n_epochs = 5
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Validation accuracy: 0.9612
1 Validation accuracy: 0.971
2 Validation accuracy: 0.9772
3 Validation accuracy: 0.9792
4 Validation accuracy: 0.9832
